<img src="View_from_Cairo_Tower_31march2007.jpg" width="200" align="left"><img src="coursera.png" width="100" align="right"><font size="6.5"><h1 align="center">Applied Data Science Capstone</font>

<h3 align="center"><font size="5">This notebook is intended for the final course in IBM Data Science Professional Certificate.</font>
<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,255);height: 3.0px;"/>

## Week 5 - Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Egypt - Giza Governorate

### Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

<a href="#item0"> 0- Add-ons Part - Install & Import Required Packages</a>

<a href="#item1"> 1- Part (1) - Build code to scrape Wikipedia page</a>

<a href="#item2"> 2- Part (2) - Build code to get the latitude and the longitude</a>

<a href="#item3"> 3- Part (3) - Explore and cluster the neighborhoods</a> 
</font>
</div>

<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item0"></a>
<h2 id="item0">0- Add-ons Part - Install & Import Required Packages</h2>

In [1]:
# Clear the output
from IPython.display import clear_output

# import Required Packages

# Check Github
try:
    from github import Github
    print ('pygithub available')
except ImportError:
    # installing folium
    print ('Installing pygithub')
    !conda install -c conda-forge pygithub
    clear_output()
    print ('pygithub Installed')
    
# Check geocoder
try:
    import pgeocode
    print ('Geocoder available')
except ImportError:
    # installing geocoder
    print ('Installing Geocoder')
    !pip install geocoder
    clear_output()
    print ('Geocoder Installed')
    
# Check folium
try:
    import folium
    print ('folium available')
except ImportError:
    # installing folium
    print ('Installing folium')
    !conda install -c conda-forge folium=0.5.0 --yes
    !pip install folium
    clear_output()
    print ('folium Installed')

# Check Esri GIS
try:
    import GIS
    print ('Esri GIS available')
except ImportError:
    # installing Esri GIS
    print ('Installing Esri GIS')
    !conda install -c esri arcgis --yes
    clear_output()
    print ('Esri GIS Installed')
    
clear_output()
print ('All Packages Correctly Installed')

All Packages Correctly Installed


In [2]:
# import Required Libraries

# import the library we use to open URLs
import urllib.request

# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

# import pandas
import pandas as pd

# convert an address into latitude and longitude values
import geocoder 

# import required libraries
from IPython.display import display
from IPython.display import HTML
from IPython.display import IFrame

# import Esri
import arcgis
import widgetsnbextension
import ipywidgets
from arcgis.gis import GIS

# import folium
import folium

# library to handle data in a vectorized manner
import numpy as np 

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# library to handle requests
import requests 
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

import os    
import sys  
import base64
import pathlib

from github import Github
from github import InputGitTreeElement

<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item1"></a>
<h2 id="item1">1. Part (1) - Build code to scrape Wikipedia page, obtain the data in the table and to transform the data into a pandas dataframe</h2>

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/Giza_Governorate, in order to obtain the data that is in the table and to transform the data into a pandas dataframe

#### Introduction - Show Study Area Map

In [3]:
# Show Map of Selected Area
# Get the Coord of Giza
g= geocoder.arcgis('Giza, EGY')

In [4]:
# create map of Giza using latitude and longitude values
map_giza = folium.Map(location=[g.lat, g.lng], zoom_start=11)
# Save it as html
map_giza.save('Giza.html')
# Optional:: Display Map
# display(map_giza)

In [5]:
%%html
<iframe src="Giza.html"width="960" height="450">

#### Read wiki page

In [6]:
# specify which URL/web page we are going to be scraping
url_wiki = "https://en.wikipedia.org/wiki/Giza_Governorate"
# open the url using urllib.request and put the HTML into the page variable
page_wiki = urllib.request.urlopen(url_wiki)
# parse the HTML from our URL into the BeautifulSoup parse tree format
bs_wiki = BeautifulSoup(page_wiki, "lxml")

# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
tables_wiki=bs_wiki.find_all("table")

In [7]:
# Get the Required table
table_Pcodes=bs_wiki.find('table', class_='sortable wikitable plainrowheaders')
table_Pcodes

<table class="sortable wikitable plainrowheaders">
<caption>
Municipal Divisions
</caption>
<tbody><tr>
<th>Anglicized name</th>
<th>Native name</th>
<th>Arabic transliteration</th>
<th>Population<br/><small>(July 2017 Est.)</small></th>
<th>Type
</th></tr>
<tr>
<td><a href="/wiki/Dokki" title="Dokki">Dokki</a></td>
<td style="text-align:right;"><span dir="rtl" lang="ar" title="Arabic language text">قسم الدقي</span></td>
<td><i lang="ar-Latn" title="Arabic-language romanization">Ad-Duqqī</i></td>
<td style="text-align:right;">71,161</td>
<td>Kism (fully urban)
</td></tr>
<tr>
<td>Pyramids</td>
<td style="text-align:right;"><span dir="rtl" lang="ar" title="Arabic language text">قسم الأهرام</span></td>
<td><i lang="ar-Latn" title="Arabic-language romanization">Al-Ahrām</i></td>
<td style="text-align:right;">661,478</td>
<td>Kism (fully urban)
</td></tr>
<tr>
<td><a href="/wiki/Agouza" title="Agouza">Agouza</a></td>
<td style="text-align:right;"><span dir="rtl" lang="ar" title="Arabic lan

In [8]:
# Lists to hold data
EnName=[]
ArName=[]
ArTrans=[]
Pop=[]
Category=[]

# Loop & ignore Borough = 'Not assigned'
for row in table_Pcodes.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0 :
        if len(cells[0].text.strip()) != 0:
            # Select Type = urban and ignore other values
            if cells[4].text.strip() == 'Kism (fully urban)':
                EnName.append(cells[0].text.strip())
                ArName.append(cells[1].text.strip())
                ArTrans.append(cells[2].text.strip())
                Pop.append(cells[3].text.strip())
                Category.append(cells[4].text.strip())  
        
# Create dataframe to hold data        
df_wiki=pd.DataFrame(EnName,columns=['Neighborhood'])
df_wiki['ArName']=ArName
df_wiki['ArTrans']=ArTrans
df_wiki['Population']=Pop
df_wiki['Category']=Category

In [9]:
# show data shape
print (df_wiki.shape)
df_wiki

(10, 5)


,Neighborhood,ArName,ArTrans,Population,Category
0,Dokki,قسم الدقي,Ad-Duqqī,"71,161",Kism (fully urban)
1,Pyramids,قسم الأهرام,Al-Ahrām,"661,478",Kism (fully urban)
2,Agouza,قسم العجوزة,Al-'Ajūzah,"279,400",Kism (fully urban)
3,Giza,قسم الجيزة,Al-Jīzah,"286,668",Kism (fully urban)
4,El Omraniya,قسم العمرانية,Al-'Umrāniyah,"367,274",Kism (fully urban)
5,Sheikh Zayed City,قسم الشيخ زايد,Ash-Shaykh Zāyid,"90,997",Kism (fully urban)
6,Bulaq,قسم بولاق الدكرور,Būlāq al-Dakrūr,"963,201",Kism (fully urban)
7,Imbaba,قسم امبابة,Imbābah,"634,686",Kism (fully urban)
8,6th of October City (2),قسم ثان 6 أكتوبر,Madīnat Sittah Uktūbar 2,"197,019",Kism (fully urban)
9,6 October City (1 & 3),قسم أول 6 أكتوبر / قسم ثالث 6 أكتوبر,Madīnat Sittah Uktūbar 1 & 3,"152,999",Kism (fully urban)


In [10]:
# Do some data enhancements for geocoding purposes
df_wiki.loc[df_wiki.ArTrans == 'Ash-Shaykh Zāyid', 'ArTrans']= 'Madinat El Saikh Zaied'

df_wiki.loc[df_wiki.Neighborhood == '6 October City (1 & 3)', 'Neighborhood']= 'Sixth of October'
df_wiki.loc[df_wiki.ArTrans == 'Madīnat Sittah Uktūbar 1 & 3', 'ArTrans']= 'Sixth of October'

df_wiki.drop(df_wiki[df_wiki.Neighborhood == '6th of October City (2)'].index, inplace=True)

In [11]:
# show data shape
print (df_wiki.shape)
df_wiki

(9, 5)


,Neighborhood,ArName,ArTrans,Population,Category
0,Dokki,قسم الدقي,Ad-Duqqī,"71,161",Kism (fully urban)
1,Pyramids,قسم الأهرام,Al-Ahrām,"661,478",Kism (fully urban)
2,Agouza,قسم العجوزة,Al-'Ajūzah,"279,400",Kism (fully urban)
3,Giza,قسم الجيزة,Al-Jīzah,"286,668",Kism (fully urban)
4,El Omraniya,قسم العمرانية,Al-'Umrāniyah,"367,274",Kism (fully urban)
5,Sheikh Zayed City,قسم الشيخ زايد,Madinat El Saikh Zaied,"90,997",Kism (fully urban)
6,Bulaq,قسم بولاق الدكرور,Būlāq al-Dakrūr,"963,201",Kism (fully urban)
7,Imbaba,قسم امبابة,Imbābah,"634,686",Kism (fully urban)
9,Sixth of October,قسم أول 6 أكتوبر / قسم ثالث 6 أكتوبر,Sixth of October,"152,999",Kism (fully urban)


<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item2"></a>
<h2 id="item2">2. Part (2) - Build code to get the latitude and the longitude coordinates</h2>

#### Using ArcGIS Geocoder

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [12]:
# Lists to hold data
Latitude=[]
Longitude=[]

# Loop & Get coord for postal codes using ArcGIS Geocoder
for i in range(0,len(df_wiki)):
    address = df_wiki['ArTrans'].iloc[i] + ', Giza, EGY'
    g= geocoder.arcgis(address)
    Latitude.append(g.lat)
    Longitude.append(g.lng)
    
# Create new dataframe to hold coord
df_Coords=df_wiki.copy()
df_Coords['Latitude']=Latitude
df_Coords['Longitude']=Longitude

In [13]:
# show data shape
print (df_Coords.shape)
df_Coords

(9, 7)


,Neighborhood,ArName,ArTrans,Population,Category,Latitude,Longitude
0,Dokki,قسم الدقي,Ad-Duqqī,"71,161",Kism (fully urban),30.03512,31.21398
1,Pyramids,قسم الأهرام,Al-Ahrām,"661,478",Kism (fully urban),30.00636,31.12200
2,Agouza,قسم العجوزة,Al-'Ajūzah,"279,400",Kism (fully urban),30.05061,31.21090
3,Giza,قسم الجيزة,Al-Jīzah,"286,668",Kism (fully urban),30.00956,31.21381
4,El Omraniya,قسم العمرانية,Al-'Umrāniyah,"367,274",Kism (fully urban),30.00243,31.20188
5,Sheikh Zayed City,قسم الشيخ زايد,Madinat El Saikh Zaied,"90,997",Kism (fully urban),30.05339,31.00164
6,Bulaq,قسم بولاق الدكرور,Būlāq al-Dakrūr,"963,201",Kism (fully urban),30.03976,31.18943
7,Imbaba,قسم امبابة,Imbābah,"634,686",Kism (fully urban),30.07625,31.20666
9,Sixth of October,قسم أول 6 أكتوبر / قسم ثالث 6 أكتوبر,Sixth of October,"152,999",Kism (fully urban),29.96630,30.93550


<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item3"></a>
<h2 id="item3">3. Part (3) - Explore and cluster the neighborhoods in Giza</h2>

Explore and cluster the neighborhoods in Giza. Just make sure:

1- to add enough Markdown cells to explain what you decided to do and to report any observations you make. 

2- to generate maps to visualize your neighborhoods and how they cluster together. 

#### Create a function to search venues to all the neighborhoods

In [14]:
# create a function to search venues to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### function to sort the venues in descending order

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Display Map

In [16]:
# Coord of Giza
GizaLat = 30.028660
GizaLng= 31.128358

# create map of Giza using latitude and longitude values
map_Giza = folium.Map(location=[GizaLat, GizaLng], zoom_start=10)

# add markers to the map
for lat, lng, label in zip(df_Coords['Latitude'], df_Coords['Longitude'], df_Coords['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Giza)

# Save it as html
map_Giza.save('Giza_Neighborhood.html')

# Optional:: Display Map
# display(map_Giza)

#### Display Resulted Maps

In [17]:
%%html
<iframe src="Giza_Neighborhood.html"width="960" height="450"></iframe>

#### Start using Foursquare API to explore the neighborhoods

In [18]:
# Read credentials from file
FourSqrFile = open("Foursquare.txt", "r")

for FourSqrlines in FourSqrFile:
    if FourSqrlines.find("CLIENT_ID") != -1:
        CLIENT_ID = str(FourSqrlines.replace("CLIENT_ID = ","")).rstrip()
    elif FourSqrlines.find("CLIENT_SECRET") != -1:
        CLIENT_SECRET = str(FourSqrlines.replace("CLIENT_SECRET = ","")).rstrip()
    elif FourSqrlines.find("VERSION") != -1:
        VERSION = str(FourSqrlines.replace("VERSION = ","")).rstrip()

FourSqrFile.close()

# Set Limit Value
LIMIT = 500

In [19]:
# let's get the venues that are within a radius of 500 meters.
df_venues = getNearbyVenues(names=df_Coords['Neighborhood'],
                                   latitudes=df_Coords['Latitude'],
                                   longitudes=df_Coords['Longitude']
                                  )

In [20]:
# check the size of the resulting dataframe
print(df_venues.shape)
df_venues

(859, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dokki,30.03512,31.21398,Seoudi (سعودي),30.036997,31.215665,Supermarket
1,Dokki,30.03512,31.21398,Sheraton Cairo Hotel & Casino,30.039195,31.220316,Hotel
2,Dokki,30.03512,31.21398,Rawi Restaurant & Bar,30.039259,31.220165,Hotel Bar
3,Dokki,30.03512,31.21398,City Drink (سيتي درينك),30.039093,31.215155,Juice Bar
4,Dokki,30.03512,31.21398,Sofitel Cairo El Gezirah,30.038709,31.224598,Hotel
5,Dokki,30.03512,31.21398,Starbucks,30.038857,31.219593,Café
6,Dokki,30.03512,31.21398,Gold’s Gym,30.033256,31.219737,Gym / Fitness Center
7,Dokki,30.03512,31.21398,McDonald's (ماكدونالدز),30.035616,31.214288,Fast Food Restaurant
8,Dokki,30.03512,31.21398,Four Seasons Hotel Cairo at First Residence,30.023935,31.217282,Hotel
9,Dokki,30.03512,31.21398,Cairo Opera House (دار الأوبرا المصرية),30.043268,31.222719,Opera House


In [21]:
# check how many venues were returned for each neighborhood
df_venues.groupby('Neighborhood').count()

# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 107 uniques categories.


In [22]:
# Save data in csv file to avoid consuming FourSquare credits
df_venues.to_csv('GizaVenues.csv')

#### Check the Sports Facilities in the returned data

In [23]:
# Select only venues with the following categories Athletics & Sports,Gym,Sports Club,Gym / Fitness Center
df_sport = df_venues.loc[(df_venues['Venue Category']=='Athletics & Sports') | (df_venues['Venue Category'] == 'Gym')
            | (df_venues['Venue Category'] == 'Sports Club') | 
                         (df_venues['Venue Category'] == 'Gym / Fitness Center')]
df_sport = df_sport.sort_values(by ='Neighborhood' )
df_sport = df_sport.reset_index(drop=True)

In [24]:
# check the size of the resulting dataframe
print(df_sport.shape)
df_sport.head(10)

(49, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agouza,30.05061,31.21090,True Gym,30.047986,31.205015,Gym / Fitness Center
1,Agouza,30.05061,31.21090,Gezira Police Club (نادي الشرطة بالجزيرة),30.047312,31.222416,Sports Club
2,Agouza,30.05061,31.21090,GSC Gym,30.055248,31.223793,Gym
3,Agouza,30.05061,31.21090,Egyptian Shooting Club (نادى الصيد المصرى),30.044372,31.201955,Sports Club
4,Agouza,30.05061,31.21090,Gezira Sporting Club (نادي الجزيرة الرياضي),30.053244,31.223966,Sports Club
5,Agouza,30.05061,31.21090,Al-Ahly Club (النادى الأهلى المصرى),30.044793,31.221417,Sports Club
6,Bulaq,30.03976,31.18943,True Gym,30.047986,31.205015,Gym / Fitness Center
7,Bulaq,30.03976,31.18943,ESC Gym,30.045362,31.199802,Gym
8,Bulaq,30.03976,31.18943,Samia Allouba Gym (سامية علوبة),30.054221,31.200915,Gym / Fitness Center
9,Bulaq,30.03976,31.18943,Al-Ahly Club (النادى الأهلى المصرى),30.044793,31.221417,Sports Club


In [25]:
# Coord of Giza
GizaLat = 30.028660
GizaLng= 31.128358

# create map of Giza using latitude and longitude values
map_Giza = folium.Map(location=[GizaLat, GizaLng], zoom_start=10)

# add markers to the map
for lat, lng, label in zip(df_sport['Venue Latitude'], df_sport['Venue Longitude'], df_sport['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Giza)

# Save it as html
map_Giza.save('Giza_SportsVenues.html')

# Optional:: Display Map
# display(map_Giza)

In [26]:
%%html
<iframe src="Giza_SportsVenues.html"width="960" height="450"></iframe>

#### Analyze Each Neighborhood

In [27]:
# one hot encoding
Giza_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Giza_onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Giza_onehot.columns[-1]] + list(Giza_onehot.columns[:-1])
Giza_onehot = Giza_onehot[fixed_columns]

# check the size of the resulting dataframe
print(Giza_onehot.shape)
Giza_onehot

(859, 108)


,Neighborhood,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,...,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theater,Track,Turkish Restaurant,Yemeni Restaurant
0,Dokki,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Dokki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
Giza_grouped = Giza_onehot.groupby('Neighborhood').mean().reset_index()
Giza_grouped

,Neighborhood,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,...,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theater,Track,Turkish Restaurant,Yemeni Restaurant
0,Agouza,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.03,0.00,0.010000,0.00,0.00,0.010000,0.01,0.00,0.00
1,Bulaq,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.03,0.01,0.010000,0.00,0.00,0.010000,0.00,0.00,0.02
2,Dokki,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,...,0.010000,0.03,0.01,0.010000,0.00,0.00,0.010000,0.00,0.01,0.00
3,El Omraniya,0.01,0.000000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.020000,...,0.010000,0.00,0.01,0.000000,0.00,0.01,0.020000,0.00,0.01,0.02
4,Giza,0.02,0.000000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.020000,...,0.010000,0.00,0.01,0.000000,0.00,0.01,0.020000,0.00,0.01,0.01
5,Imbaba,0.02,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.04,0.00,0.000000,0.00,0.01,0.000000,0.00,0.00,0.00
6,Pyramids,0.00,0.012048,0.012048,0.000000,0.00,0.012048,0.012048,0.012048,0.012048,...,0.012048,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00
7,Sheikh Zayed City,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.030000,...,0.020000,0.02,0.00,0.020000,0.01,0.00,0.000000,0.00,0.00,0.00
8,Sixth of October,0.00,0.000000,0.000000,0.013158,0.00,0.000000,0.000000,0.000000,0.013158,...,0.039474,0.00,0.00,0.092105,0.00,0.00,0.013158,0.00,0.00,0.00


Let's confirm the new size & print each neighborhood along with the top 10 most common venues

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Giza_grouped['Neighborhood']

for ind in np.arange(Giza_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Giza_grouped.iloc[ind, :], num_top_venues)

# check the size of the resulting dataframe
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(9, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agouza,Café,Hotel,Italian Restaurant,Restaurant,Sports Club,Seafood Restaurant,Bakery,Middle Eastern Restaurant,Burger Joint,Sushi Restaurant
1,Bulaq,Café,Seafood Restaurant,Gym / Fitness Center,Dessert Shop,Hotel,Italian Restaurant,Middle Eastern Restaurant,Lebanese Restaurant,Sports Club,Juice Bar
2,Dokki,Café,Hotel,Restaurant,Italian Restaurant,Sports Club,Hotel Bar,Gym / Fitness Center,Lounge,Lebanese Restaurant,Nightclub
3,El Omraniya,Café,Hotel Bar,Italian Restaurant,Hotel,Restaurant,Gym / Fitness Center,Nightclub,Historic Site,Food Court,Dessert Shop
4,Giza,Café,Hotel,Hotel Bar,Dessert Shop,Italian Restaurant,Lounge,Food Court,Gym / Fitness Center,Historic Site,Nightclub
5,Imbaba,Café,Hotel,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Lebanese Restaurant,Egyptian Restaurant,Dessert Shop,Coffee Shop,Bar
6,Pyramids,Café,Historic Site,Hotel,Falafel Restaurant,Convenience Store,Restaurant,Stables,History Museum,Scenic Lookout,Burger Joint
7,Sheikh Zayed City,Café,Shopping Mall,Coffee Shop,Food Court,Sports Club,Bakery,Burger Joint,Middle Eastern Restaurant,Gym / Fitness Center,Convenience Store
8,Sixth of October,Café,Syrian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Sports Club,Coffee Shop,Shopping Mall,Supermarket,Kebab Restaurant,Restaurant


#### Cluster Neighborhoods

In [32]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 3

Giza_grouped_clustering = Giza_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Giza_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 2, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Giza_merged = df_Coords

# merge Giza_grouped with Giza_data to add latitude/longitude for each neighborhood
Giza_merged = Giza_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [34]:
# check the size of the resulting dataframe
print(Giza_merged.shape)
Giza_merged

(9, 18)


,Neighborhood,ArName,ArTrans,Population,Category,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dokki,قسم الدقي,Ad-Duqqī,"71,161",Kism (fully urban),30.03512,31.21398,1,Café,Hotel,Restaurant,Italian Restaurant,Sports Club,Hotel Bar,Gym / Fitness Center,Lounge,Lebanese Restaurant,Nightclub
1,Pyramids,قسم الأهرام,Al-Ahrām,"661,478",Kism (fully urban),30.00636,31.12200,2,Café,Historic Site,Hotel,Falafel Restaurant,Convenience Store,Restaurant,Stables,History Museum,Scenic Lookout,Burger Joint
2,Agouza,قسم العجوزة,Al-'Ajūzah,"279,400",Kism (fully urban),30.05061,31.21090,1,Café,Hotel,Italian Restaurant,Restaurant,Sports Club,Seafood Restaurant,Bakery,Middle Eastern Restaurant,Burger Joint,Sushi Restaurant
3,Giza,قسم الجيزة,Al-Jīzah,"286,668",Kism (fully urban),30.00956,31.21381,1,Café,Hotel,Hotel Bar,Dessert Shop,Italian Restaurant,Lounge,Food Court,Gym / Fitness Center,Historic Site,Nightclub
4,El Omraniya,قسم العمرانية,Al-'Umrāniyah,"367,274",Kism (fully urban),30.00243,31.20188,1,Café,Hotel Bar,Italian Restaurant,Hotel,Restaurant,Gym / Fitness Center,Nightclub,Historic Site,Food Court,Dessert Shop
5,Sheikh Zayed City,قسم الشيخ زايد,Madinat El Saikh Zaied,"90,997",Kism (fully urban),30.05339,31.00164,0,Café,Shopping Mall,Coffee Shop,Food Court,Sports Club,Bakery,Burger Joint,Middle Eastern Restaurant,Gym / Fitness Center,Convenience Store
6,Bulaq,قسم بولاق الدكرور,Būlāq al-Dakrūr,"963,201",Kism (fully urban),30.03976,31.18943,1,Café,Seafood Restaurant,Gym / Fitness Center,Dessert Shop,Hotel,Italian Restaurant,Middle Eastern Restaurant,Lebanese Restaurant,Sports Club,Juice Bar
7,Imbaba,قسم امبابة,Imbābah,"634,686",Kism (fully urban),30.07625,31.20666,1,Café,Hotel,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Lebanese Restaurant,Egyptian Restaurant,Dessert Shop,Coffee Shop,Bar
9,Sixth of October,قسم أول 6 أكتوبر / قسم ثالث 6 أكتوبر,Sixth of October,"152,999",Kism (fully urban),29.96630,30.93550,0,Café,Syrian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Sports Club,Coffee Shop,Shopping Mall,Supermarket,Kebab Restaurant,Restaurant


Finally, let's visualize the resulting clusters

In [35]:
# Coord of Giza
GizaLat = 30.028660
GizaLng= 31.128358

# create map of Giza using latitude and longitude values
map_clusters = folium.Map(location=[GizaLat, GizaLng], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

# add markers to map      
for lat, lon, poi, cluster in zip(Giza_merged['Latitude'], Giza_merged['Longitude'], Giza_merged['Neighborhood'], Giza_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
# Save it as html
map_clusters.save('Giza_Clusters.html')

# Optional:: Display Map
# display(map_clusters)

#### Display Resulted Maps

In [36]:
%%html
<iframe src="Giza_Clusters.html"width="475" height="350"></iframe><iframe src="Giza_SportsVenues.html" 
width="475" height="350"></iframe>

#### Examine Clusters

Cluster 1 ::: South Giza

In [37]:
Giza_merged.loc[Giza_merged['Cluster Labels'] == 0, 
                   Giza_merged.columns[[1] + list(range(5, Giza_merged.shape[1]))]]

,ArName,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,قسم الشيخ زايد,30.05339,31.00164,0,Café,Shopping Mall,Coffee Shop,Food Court,Sports Club,Bakery,Burger Joint,Middle Eastern Restaurant,Gym / Fitness Center,Convenience Store
9,قسم أول 6 أكتوبر / قسم ثالث 6 أكتوبر,29.96630,30.93550,0,Café,Syrian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Sports Club,Coffee Shop,Shopping Mall,Supermarket,Kebab Restaurant,Restaurant


Cluster 2 ::: Downtown Giza

In [38]:
Giza_merged.loc[Giza_merged['Cluster Labels'] == 1, 
                   Giza_merged.columns[[1] + list(range(5, Giza_merged.shape[1]))]]

,ArName,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,قسم الدقي,30.03512,31.21398,1,Café,Hotel,Restaurant,Italian Restaurant,Sports Club,Hotel Bar,Gym / Fitness Center,Lounge,Lebanese Restaurant,Nightclub
2,قسم العجوزة,30.05061,31.21090,1,Café,Hotel,Italian Restaurant,Restaurant,Sports Club,Seafood Restaurant,Bakery,Middle Eastern Restaurant,Burger Joint,Sushi Restaurant
3,قسم الجيزة,30.00956,31.21381,1,Café,Hotel,Hotel Bar,Dessert Shop,Italian Restaurant,Lounge,Food Court,Gym / Fitness Center,Historic Site,Nightclub
4,قسم العمرانية,30.00243,31.20188,1,Café,Hotel Bar,Italian Restaurant,Hotel,Restaurant,Gym / Fitness Center,Nightclub,Historic Site,Food Court,Dessert Shop
6,قسم بولاق الدكرور,30.03976,31.18943,1,Café,Seafood Restaurant,Gym / Fitness Center,Dessert Shop,Hotel,Italian Restaurant,Middle Eastern Restaurant,Lebanese Restaurant,Sports Club,Juice Bar
7,قسم امبابة,30.07625,31.20666,1,Café,Hotel,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Lebanese Restaurant,Egyptian Restaurant,Dessert Shop,Coffee Shop,Bar


Cluster 3 ::: East & West Giza

In [39]:
Giza_merged.loc[Giza_merged['Cluster Labels'] == 2, 
                   Giza_merged.columns[[1] + list(range(5, Giza_merged.shape[1]))]]

,ArName,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,قسم الأهرام,30.00636,31.122,2,Café,Historic Site,Hotel,Falafel Restaurant,Convenience Store,Restaurant,Stables,History Museum,Scenic Lookout,Burger Joint


## Observation

<div class="alert alert-block alert-info" style="margin-top: 20px">  
<font size = 3>
Cluster 2 ::: is the selected Cluster for the existance of most of sports facilities and varaities of venues.
</font>
</div>

In [41]:
%%html
<iframe src="Giza_Clusters.html"width="475" height="350"></iframe><iframe src="Giza_SportsVenues.html" 
width="475" height="350"></iframe>

#### Create a function to publish on Github

In [46]:
# Read credentials from file
GitHubFile = open("GitHub.txt", "r")

for GitHublines in GitHubFile:
    if GitHublines.find("USER") != -1:
        GitUSER = str(GitHublines.replace("USER = ","")).rstrip()
    elif GitHublines.find("PASSWORD") != -1:
        GitPass = str(GitHublines.replace("PASSWORD = ","")).rstrip()
    elif GitHublines.find("REPO") != -1:
        GitRepo = str(GitHublines.replace("REPO = ","")).rstrip()

GitHubFile.close()

In [47]:
def publishGitHub(filelist,filenames,msg):
    #Connect to Github
    g = Github(GitUSER,GitPass)
    repo = g.get_user().get_repo(GitRepo)
     
    # File list to publish and publish names
    file_list = filelist
    file_names = filenames
    commit_message = msg
    master_ref = repo.get_git_ref('heads/master')
    master_sha = master_ref.object.sha
    base_tree = repo.get_git_tree(master_sha)
    element_list = list()
    # Loop in list
    for i, entry in enumerate(file_list):
        with open(entry) as input_file:
            data = input_file.read()
        if entry.endswith('.png'):
            data = base64.b64encode(data)
        element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
        element_list.append(element)
    tree = repo.create_git_tree(element_list, base_tree)
    parent = repo.get_git_commit(master_sha)
    commit = repo.create_git_commit(commit_message, tree, [parent])
    master_ref.edit(commit.sha)

In [ ]:
# Publish Part 3 on GitHub
filelist = ['CT-Week5.ipynb']
filenames = ['CT-Week5.ipynb']
msg='Capstone Week(5)'
publishGitHub(filelist,filenames,msg)

<hr>

Copyright &copy; Ahmed Reda 2019.